In [1]:
import torch
torch.__version__

'2.0.0+cu117'

In [2]:
torch.cuda.is_available()

True

In [ ]:
python -m torch.distributed.launch --master_port 14528 --nproc_per_node=1 dinov2/train/train.py --config-file=dinov2/configs/train/vitl16_short.yaml --output-dir=./output/ train.dataset_path=TileDataset:split=TRAIN:root=/yuhaowang/data/processed_data/private_chunk_7/output

In [2]:
from PIL import ImageFile, Image
import os
from tqdm import tqdm
from pathlib import Path
import shutil
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:

data_path='/yuhaowang/data/processed_data/private_chunk_7'
image_paths=Path(data_path).rglob('*.png')

def validate_img(path):
    try:
        img = Image.open(path).convert(mode="RGB")
    except:
        os.remove(path)
        print('rm rf ',path)
        pass
    return 


In [4]:
len(list(image_paths))

291931

In [ ]:
for img_path in image_paths:
    validate_img(img_path)

rm rf  /yuhaowang/data/processed_data/private_chunk_7/output/S24570.svs/23296x_95104y.png
rm rf  /yuhaowang/data/processed_data/private_chunk_7/output/S25932.svs/130048x_37632y.png
rm rf  /yuhaowang/data/processed_data/private_chunk_7/output/S26768.svs/199936x_31104y.png
rm rf  /yuhaowang/data/processed_data/private_chunk_7/output/S22457.svs/120485x_97052y.png
rm rf  /yuhaowang/data/processed_data/private_chunk_7/output/S25618.svs/146212x_11297y.png
rm rf  /yuhaowang/data/processed_data/private_chunk_7/output/S25618.svs/41764x_89121y.png


In [12]:
import torch
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "0"
torch.cuda.set_device(0)

In [ ]:
print("当前cuda设备是", torch.cuda.current_device()) 

当前cuda设备是 0


: 

In [ ]:
import torch
print("当前cuda设备是", torch.cuda.current_device()) # 获取当前cuda设备
 
a=torch.tensor([1, 2, 3, 4, 5], device='cuda') # 在当前cuda设备上创建一个张量
print(a)
 
device=torch.device('cuda')                    # 创建一个当前cuda设备
 
b=torch.tensor([1, 2, 3, 4, 5], device=device) # 在当前cuda设备上创建一个张量
print(b)
 
c=torch.tensor([1, 2, 3, 4, 5]).to(device)     # 将张量移动到当前cuda设备
print(c)
 
d=torch.tensor([1, 2, 3, 4, 5]).to('cuda')     # 将张量移动到当前cuda设备
print(d)
 
e=torch.tensor([1, 2, 3, 4, 5]).cuda()         # 将张量移动到当前cuda设备
print(e)
 
f=torch.tensor([1, 2, 3, 4, 5]).cuda(device)   # 将张量移动到当前cuda设备
print(f)
 


In [3]:
import torch
torch.cuda.device_count()

1

In [3]:
from fastai.vision.all import Path, get_image_files, verify_images

from typing import Any, Optional, Callable, Tuple

from PIL import Image
from pathlib import Path
from dinov2.data.datasets.extended import ExtendedVisionDataset

from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True


In [2]:
root='/yuhaowang/data/private_chunk_4'
root = Path(root).expanduser()
image_paths = get_image_files(root)

NameError: name 'Path' is not defined

In [4]:
len(image_paths)

290868

In [1]:
from multiprocessing import Pool, cpu_count
cpu_count()

128

In [7]:
data=Image.open('/yuhaowang/data/private_chunk_4/output/S04168.svs/16352x_27353y.png').convert(mode="RGB")

In [26]:
type(data)

PIL.Image.Image

In [11]:
def default_image(size=(256, 256)):
    return torch.zeros(3, *size)

In [14]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
#
# This source code is licensed under the Apache License, Version 2.0
# found in the LICENSE file in the root directory of this source tree.

import logging

from torchvision import transforms

# Copyright (c) Meta Platforms, Inc. and affiliates.
#
# This source code is licensed under the Apache License, Version 2.0
# found in the LICENSE file in the root directory of this source tree.

from typing import Sequence

import torch
from torchvision import transforms


class GaussianBlur(transforms.RandomApply):
    """
    Apply Gaussian Blur to the PIL image.
    """

    def __init__(self, *, p: float = 0.5, radius_min: float = 0.1, radius_max: float = 2.0):
        # NOTE: torchvision is applying 1 - probability to return the original image
        keep_p = 1 - p
        transform = transforms.GaussianBlur(kernel_size=9, sigma=(radius_min, radius_max))
        super().__init__(transforms=[transform], p=keep_p)


class MaybeToTensor(transforms.ToTensor):
    """
    Convert a ``PIL Image`` or ``numpy.ndarray`` to tensor, or keep as is if already a tensor.
    """

    def __call__(self, pic):
        """
        Args:
            pic (PIL Image, numpy.ndarray or torch.tensor): Image to be converted to tensor.
        Returns:
            Tensor: Converted image.
        """
        if isinstance(pic, torch.Tensor):
            return pic
        return super().__call__(pic)


# Use timm's names
IMAGENET_DEFAULT_MEAN = (0.485, 0.456, 0.406)
IMAGENET_DEFAULT_STD = (0.229, 0.224, 0.225)


def make_normalize_transform(
    mean: Sequence[float] = IMAGENET_DEFAULT_MEAN,
    std: Sequence[float] = IMAGENET_DEFAULT_STD,
) -> transforms.Normalize:
    return transforms.Normalize(mean=mean, std=std)


# This roughly matches torchvision's preset for classification training:
#   https://github.com/pytorch/vision/blob/main/references/classification/presets.py#L6-L44
def make_classification_train_transform(
    *,
    crop_size: int = 224,
    interpolation=transforms.InterpolationMode.BICUBIC,
    hflip_prob: float = 0.5,
    mean: Sequence[float] = IMAGENET_DEFAULT_MEAN,
    std: Sequence[float] = IMAGENET_DEFAULT_STD,
):
    transforms_list = [transforms.RandomResizedCrop(crop_size, interpolation=interpolation)]
    if hflip_prob > 0.0:
        transforms_list.append(transforms.RandomHorizontalFlip(hflip_prob))
    transforms_list.extend(
        [
            MaybeToTensor(),
            make_normalize_transform(mean=mean, std=std),
        ]
    )
    return transforms.Compose(transforms_list)


# This matches (roughly) torchvision's preset for classification evaluation:
#   https://github.com/pytorch/vision/blob/main/references/classification/presets.py#L47-L69
def make_classification_eval_transform(
    *,
    resize_size: int = 256,
    interpolation=transforms.InterpolationMode.BICUBIC,
    crop_size: int = 224,
    mean: Sequence[float] = IMAGENET_DEFAULT_MEAN,
    std: Sequence[float] = IMAGENET_DEFAULT_STD,
) -> transforms.Compose:
    transforms_list = [
        transforms.Resize(resize_size, interpolation=interpolation),
        transforms.CenterCrop(crop_size),
        MaybeToTensor(),
        make_normalize_transform(mean=mean, std=std),
    ]
    return transforms.Compose(transforms_list)

logger = logging.getLogger("dinov2")


class DataAugmentationDINO(object):
    def __init__(
        self,
        global_crops_scale,
        local_crops_scale,
        local_crops_number,
        global_crops_size=224,
        local_crops_size=96,
    ):
        self.global_crops_scale = global_crops_scale
        self.local_crops_scale = local_crops_scale
        self.local_crops_number = local_crops_number
        self.global_crops_size = global_crops_size
        self.local_crops_size = local_crops_size

        logger.info("###################################")
        logger.info("Using data augmentation parameters:")
        logger.info(f"global_crops_scale: {global_crops_scale}")
        logger.info(f"local_crops_scale: {local_crops_scale}")
        logger.info(f"local_crops_number: {local_crops_number}")
        logger.info(f"global_crops_size: {global_crops_size}")
        logger.info(f"local_crops_size: {local_crops_size}")
        logger.info("###################################")

        # random resized crop and flip
        self.geometric_augmentation_global = transforms.Compose(
            [
                transforms.RandomResizedCrop(
                    global_crops_size, scale=global_crops_scale, interpolation=transforms.InterpolationMode.BICUBIC
                ),
                transforms.RandomHorizontalFlip(p=0.5),
            ]
        )

        self.geometric_augmentation_local = transforms.Compose(
            [
                transforms.RandomResizedCrop(
                    local_crops_size, scale=local_crops_scale, interpolation=transforms.InterpolationMode.BICUBIC
                ),
                transforms.RandomHorizontalFlip(p=0.5),
            ]
        )

        # color distorsions / blurring
        color_jittering = transforms.Compose(
            [
                transforms.RandomApply(
                    [transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)],
                    p=0.8,
                ),
                transforms.RandomGrayscale(p=0.2),
            ]
        )

        global_transfo1_extra = GaussianBlur(p=1.0)

        global_transfo2_extra = transforms.Compose(
            [
                GaussianBlur(p=0.1),
                transforms.RandomSolarize(threshold=128, p=0.2),
            ]
        )

        local_transfo_extra = GaussianBlur(p=0.5)

        # normalization
        self.normalize = transforms.Compose(
            [
                transforms.ToTensor(),
                make_normalize_transform(),
            ]
        )

        self.global_transfo1 = transforms.Compose([color_jittering, global_transfo1_extra, self.normalize])
        self.global_transfo2 = transforms.Compose([color_jittering, global_transfo2_extra, self.normalize])
        self.local_transfo = transforms.Compose([color_jittering, local_transfo_extra, self.normalize])

    def __call__(self, image):
        output = {}

        # global crops:
        im1_base = self.geometric_augmentation_global(image)
        global_crop_1 = self.global_transfo1(im1_base)

        im2_base = self.geometric_augmentation_global(image)
        global_crop_2 = self.global_transfo2(im2_base)

        output["global_crops"] = [global_crop_1, global_crop_2]

        # global crops for teacher:
        output["global_crops_teacher"] = [global_crop_1, global_crop_2]

        # local crops:
        local_crops = [
            self.local_transfo(self.geometric_augmentation_local(image)) for _ in range(self.local_crops_number)
        ]
        output["local_crops"] = local_crops
        output["offsets"] = ()

        return output


In [27]:
def get_target( index: int) -> Any:
    return 0

image=default_image()
target = get_target(15000)
data_transform = DataAugmentationDINO(
    global_crops_scale=[0.32,1.0],
    local_crops_scale=[0.05,0.32],
    local_crops_number=8
)
image, target = data_transform(image)



/opt/conda/envs/dinov2/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
from PIL import Image

# 指定图像大小 (宽, 高)
size = 
# 指定背景颜色，例如白色 (255, 255, 255) 或黑色 (0, 0, 0)
background_color =

# 创建空白图像


# 保存或显示图像
blank_image.show()  


In [ ]:
yuhao/project/FMBC/dinov2/output/eval/training_399999/teacher_checkpoint.pth

In [3]:
import os
len(os.listdir('/home/yuhaowang/data/TCGA-BRCA/output'))

1133

In [5]:
len(os.listdir('/data4/embedding/TCGA-BRCA/FMBC'))

1133

In [1]:
import os
data_dir = '/home/yuhaowang/data'
processed_dir = '/data4/embedding'
all_datasets = os.listdir(data_dir)
processed_datasets = os.listdir(processed_dir) if os.path.exists(processed_dir) else []
unprocessed_datasets = []
for dataset in all_datasets:
    if len(os.listdir(os.path.join(data_dir, dataset,'output'))) != len(os.listdir(os.path.join(processed_dir, dataset,'FMBC'))):

        unprocessed_datasets.append(dataset)
#unprocessd_dataset= [d for d in all_datasets if len(os.listdir(os.path.join(data_dir, d,'output'))) != len(os.listdir(os.path.join(processed_dir, d)))]

In [2]:
unprocessed_datasets

['TCGA-BRCA',
 'ACROBAT',
 'private_chunk_7',
 'CMB-BRCA',
 'private_chunk_9',
 'BRACS',
 'TUPAC',
 'AIDPATH',
 'private_chunk_4',
 'Multi-omic',
 'BreakHis',
 'private_chunk_8',
 'BCNB',
 'private_chunk_6',
 'SLN-Breast',
 'HyReCo',
 'private_chunk_1',
 'private_chunk_5',
 'IMPRESS',
 'TIGER',
 'CAMELYON17',
 'private_chunk_3',
 'Post-NAT-BRCA',
 'HE-vs-MPM',
 'GTEX_Breast',
 'CPTAC-BREAST-all',
 'CAMELYON16',
 'private_chunk_2',
 'private_chunk_10']